In [1]:
import os
import json
import traceback
import asyncio

from datetime import datetime

from tqdm.notebook import tqdm

from langchain_mcp_adapters.client import MultiServerMCPClient

from react_agent.src.agents.react_agent import ReActAgent
from react_agent.src.agents.triage import Triage

from react_agent.src.config.system_parameters import TriageSettings

from react_agent.src.util.tools_fabric import ToolsFabric

from experiments.models.experiment_models import LabeledQAPairFacts, ExperimentResult
from experiments.metrics.fact_score.fact_scorer import FactScorer
from experiments.metrics.bert_score import BertScore
from experiments.metrics.agent_judge import AgentJudgeEvaluator

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
USE_MCP = True
DEBUG_MODE = False
USE_TRIAGE = False

AGENT_JUDGE_MODEL = "anthropic--claude-3.5-sonnet"

fact_scorer = FactScorer()
path_to_source_data = "./resources/02_facts/"

date_str = datetime.now().strftime("%d%m%Y")
path_to_results = f"./results/{date_str}"

if not os.path.exists(path_to_results):
    os.makedirs(path_to_results)

time_str = datetime.now().strftime("%H%M%S")

result_file_name = f"{path_to_results}/{time_str}_results.json"
failed_experiments_file_name = f"{path_to_results}/{time_str}_failed_experiments.json"

In [4]:
file_path = os.path.abspath(path_to_source_data + "data_set_facts.json")

with open(file_path, encoding="utf8") as f:
    data = json.load(f)
    data_set = [LabeledQAPairFacts(**item) for item in data]

In [5]:
data_set_for_experiment = data_set

In [6]:
triage_agent = Triage()

In [7]:
async def run_experiment(data_row):
    result = ExperimentResult(
        id=data_row.id,
        question=data_row.question,
        answer=data_row.answer,
        product=data_row.product,
        category=data_row.category,
        persona=data_row.persona,
        activity=data_row.activity,
        country=data_row.country,
    )
    print(f"Processing question with id: {data_row.id}")
    if USE_TRIAGE:
        print("\tUsing Triage to categorize the query...")
        query_categorization = triage_agent.triage_user_message(
            user_message=data_row.question
        )
        if DEBUG_MODE:
            print(f"\tQuery categorization: {query_categorization}")

    result.triage_category = (
        query_categorization["category"]
        if USE_TRIAGE
        else TriageSettings.Categories.ALL
    )

    print(f"\tFetching tools for category: {result.triage_category}...")
    tools = ToolsFabric.get_tools_for_category(
        use_mcp=USE_MCP,
        configuration=result.triage_category,
    )

    print("\tAsking agent question...")
    if USE_MCP:
        if USE_MCP:
            try:
                # Set a reasonable timeout for the entire MCP client interaction
                # Adjust the 300 seconds (5 minutes) as needed
                async with asyncio.timeout(300):
                    async with MultiServerMCPClient(tools) as client:
                        agent = ReActAgent(tool_list=client.get_tools())
                        await agent.arun_agent_with_input(
                            user_message=data_row.question,
                            debug=DEBUG_MODE,
                        )
            except asyncio.TimeoutError:
                print(
                    f"ERROR: MultiServerMCPClient operation timed out for question ID: {data_row.id}"
                )
                # You might want to log this, set a specific result status,
                # or raise a custom exception here.
                return result  # Or handle it as appropriate for your experiment
            except Exception as e:
                print(
                    f"ERROR: An unexpected error occurred with MultiServerMCPClient for question ID {data_row.id}: {e}"
                )
                raise  # Re-raise if you want the exception to propagate
    else:
        agent = ReActAgent(tool_list=tools)
        agent.run_agent_with_input(user_message=data_row.question, debug=DEBUG_MODE)

    run_data = agent.get_execution_data()

    result.tools_used = run_data.tools_used
    result.tool_calls_count = len(run_data.tools_used)
    result.excecution_time_seconds = run_data.excecution_time_seconds
    result.model_used = run_data.model_used
    result.tokens_consumed = run_data.tokens_consumed
    result.llm_call_count = run_data.llm_call_count
    result.facts = data_row.facts
    result.generated_answer = run_data.final_output

    print("\tFinished agent execution")

    print("\tCalculating FactScore...")
    result.fact_score = await fact_scorer.get_fact_score(
        facts=data_row.facts,
        knowledge_source=result.generated_answer,
        debug=DEBUG_MODE,
    )

    print("\tCalculating BERTScore...")
    result.bert_score = BertScore.compute_score(
        expected_response=data_row.answer, actual_response=result.generated_answer
    )

    print("\tEvaluating agent response with Agent as a Judge...")
    result.agent_judge_model = AGENT_JUDGE_MODEL
    agent_evaluator = AgentJudgeEvaluator(model=AGENT_JUDGE_MODEL)

    try:
        agent_judge_outcome = agent_evaluator.evaluate(
            question=data_row.question,
            expert_answer=data_row.answer,
            generated_answer=result.generated_answer,
        )
    except Exception as e:
        print(f"\tAn error occurred during agent evaluation: {e}")
        print("\t Trying to run evaluation again...")
        agent_judge_outcome = agent_evaluator.evaluate(
            question=data_row.question,
            expert_answer=data_row.answer,
            generated_answer=result.generated_answer,
        )

    result.agent_judge_outcome = agent_judge_outcome.answer
    result.agent_judge_reasoning = agent_judge_outcome.reasoning

    result.agent_judge_call_count = agent_evaluator.get_llm_call_count()
    result.agent_judge_tokens_consumed = agent_evaluator.get_token_consumption()

    print(f"\tFinished experiment for id: {data_row.id}\n")

    return result

In [8]:
experiment_results: list[ExperimentResult] = []
failed_experiments = []

In [13]:
for data_row in tqdm(data_set_for_experiment, desc="Running cases", unit="case"):
    if data_row.id in [result.id for result in experiment_results]:
        print(f"Skipping already processed id: {data_row.id}")
        continue
    try:
        result = await run_experiment(data_row)

        experiment_results.append(result)
    except Exception as e:
        failed_experiments.append(
            {
                "id": data_row.id,
                "error": str(e),
                "traceback": traceback.format_exc(),
            }
        )

Running cases:   0%|          | 0/91 [00:00<?, ?case/s]

Skipping already processed id: EA-1
Skipping already processed id: EA-2
Skipping already processed id: EA-3
Skipping already processed id: EA-4
Skipping already processed id: EA-5
Skipping already processed id: EA-6
Skipping already processed id: EA-7
Skipping already processed id: EA-8
Skipping already processed id: EA-9
Skipping already processed id: EA-10
Skipping already processed id: EA-11
Skipping already processed id: EA-12
Skipping already processed id: EA-13
Skipping already processed id: EA-14
Skipping already processed id: EA-15
Skipping already processed id: EA-16
Skipping already processed id: EA-17
Skipping already processed id: EA-18
Skipping already processed id: EA-19
Skipping already processed id: EA-20
Skipping already processed id: EA-21
Skipping already processed id: EA-22
Skipping already processed id: EA-23
Skipping already processed id: EA-24
Skipping already processed id: EA-25
Skipping already processed id: EA-26
Skipping already processed id: EA-27
Skipping a

In [14]:
# Serialize experiment results to list of dicts
import pandas as pd

records = [r.model_dump(mode="json") for r in experiment_results]
# Make it a dataframe in order to flatten nested structures
df = pd.json_normalize(records)

df

,id,question,answer,product,category,persona,activity,country,facts,bert_score,...,fact_score.supporting_fact_score,fact_score.combined_fact_score,fact_score.direct_facts,fact_score.supporting_facts,agent_judge_tokens_consumed.input_tokens,agent_judge_tokens_consumed.output_tokens,agent_judge_tokens_consumed.total_tokens,tokens_consumed.input_tokens,tokens_consumed.output_tokens,tokens_consumed.total_tokens
0,EA-1,How can I attach additional files to standard ...,To attach additional files to the standard XML...,ERP (only SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,Cross,[{'fact': 'You can attach additional files to ...,0.206500,...,0.8,0.750000,[{'fact': 'You can attach additional files to ...,[{'fact': 'The enhancement is implemented in A...,7094,3,7097,16451,2529,18980
1,EA-3,For a Greece customer invoice process in Publi...,"Yes, you can enhance or extend the generated X...",ERP (only SAP S/4HANA Cloud Public Edition),How to Develop (Maintenance - Customer case su...,Developer (Customer side),Custom Development,GR,[{'fact': 'You can enhance or extend the gener...,0.169638,...,0.0,0.333333,[{'fact': 'You can enhance or extend the gener...,[{'fact': 'The method SET_OUTPUT_DATA belongs ...,8731,3,8734,20693,2959,23652
2,EA-4,"As a Public Cloud customer in Spain, can I ext...","Yes, as a Public Cloud customer in Spain, you ...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,ES,[{'fact': 'A Public Cloud customer in Spain ca...,0.282810,...,0.5,0.750000,[{'fact': 'A Public Cloud customer in Spain ca...,[{'fact': 'It is possible to define that for s...,6358,3,6361,5716,624,6340
3,EA-5,"As a Public Cloud customer in Italy, can I ext...","Yes, as a Public Cloud customer in Italy, you...",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,IT,[{'fact': 'As a Public Cloud customer in Italy...,0.298782,...,1.0,1.000000,[{'fact': 'As a Public Cloud customer in Italy...,[{'fact': 'You can define that for some types ...,6809,3,6812,16174,2115,18289
4,EA-7,Can I extend an Action in eDocument Cockpit fo...,"No, Actions can not be extended in Public Cloud",ERP (only SAP S/4HANA Cloud Public Edition),Documentation (Maintenance - Customer case sup...,Developer (Customer side),Custom Development,Cross,[{'fact': 'Actions cannot be extended in Publi...,0.105028,...,0.0,0.000000,[{'fact': 'Actions cannot be extended in Publi...,[],11970,530,12500,7891,2141,10032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,STC-1,Which steps do I have to do in order to enable...,To create eDocuments for Settlement Management...,ERP,Analysis (Development),Architects (Software vendor side),Design,FR,[{'fact': 'Settlement Management Documents mus...,0.162307,...,0.0,0.000000,[{'fact': 'Settlement Management Documents mus...,[{'fact': 'Example entries for the mapping cla...,18471,3,18474,13833,2762,16595
87,STC-4,Which steps do I have to do in order to enable...,In order to trigger the mapping class you have...,ERP,Analysis (Development),Architects (Software vendor side),Design,FR,[{'fact': 'Entries must be inserted into the v...,0.219856,...,0.0,0.000000,[{'fact': 'Entries must be inserted into the v...,[{'fact': 'These steps are required to trigger...,14548,666,15214,14233,2649,16882
88,STC-5,"I want to see, how the enablement for source t...",First you need to make the Settlement Manageme...,ERP,Coding (Development),Developer (Software vendor side),Development,FR,[{'fact': 'The enablement for source type 'Set...,0.555460,...,1.0,1.000000,[{'fact': 'The enablement for source type 'Set...,"[{'fact': 'If mo_adaptor is bound, the method ...",12165,3,12168,12354,763,13117
89,STC-18,Which methods of the France B2B eInvoice eDocu...,- method IS_RELEVANT of class CL_ED

In [15]:
# Convert to list of dictionaries (JSON objects)
json_list = df.to_dict(orient="records")

with open(result_file_name, "w") as f:
    json.dump(json_list, f, indent=2)